In [2]:
import pandas as pd
import numpy as np
import os
%config IPCompleter.greedy=True

In [3]:
OUTPUT_DIR = 'outputs/'
INPUT_DIR = 'data/'

HOD_same = 'HumanObserved-Dataset/same_pairs.csv'
HOD_diff = 'HumanObserved-Dataset/diffn_pairs.csv'
HOD_features = 'HumanObserved-Dataset/HumanObserved-Features-Data.csv'    

GSC_same = 'GSC-Dataset/same_pairs.csv'
GSC_diff = 'GSC-Dataset/same_pairs.csv'
GSC_features = 'GSC-Dataset/GSC_Features.csv'

In [100]:
category_dict = {
    'hod' : [HOD_same, HOD_diff, HOD_features],
    'gsc' : [GSC_same, GSC_diff, GSC_features],
}
same, diff, features = category_dict['hod']

same_data = pd.read_csv(os.path.join(INPUT_DIR, same), sep=',', nrows = 1000)
diff_data = pd.read_csv(os.path.join(INPUT_DIR, diff),  sep=',', nrows = 1000)
features_data = pd.read_csv(os.path.join(INPUT_DIR, features), sep=',')

# print(same_data.columns.values, diff_data.columns.values, features_data.columns.values)
print(same_data.shape, diff_data.shape, features_data.shape)
merged_data = pd.concat([same_data, diff_data])
# print(merged_data.shape)
temp = pd.merge(merged_data, features_data, left_on='img_id_A', right_on='img_id', how='left')
temp = pd.merge(temp, features_data, left_on='img_id_B', right_on='img_id', how='left', suffixes = ('_a', '_b'))
temp = temp.drop(columns = ['img_id_a', 'img_id_b', 'Unnamed: 0_a', 'Unnamed: 0_b'])
raw_data = temp.iloc[:,3:]
raw_target = temp["target"]
# print(temp.shape)
print(temp.columns.values)
# print(raw_target, raw_data)
# temp = pd.merge(same_data, diff_data)
# print(features_data[3:9])

(791, 3) (1000, 3) (1026, 11)
['img_id_A' 'img_id_B' 'target' 'f1_a' 'f2_a' 'f3_a' 'f4_a' 'f5_a' 'f6_a'
 'f7_a' 'f8_a' 'f9_a' 'f1_b' 'f2_b' 'f3_b' 'f4_b' 'f5_b' 'f6_b' 'f7_b'
 'f8_b' 'f9_b']


In [93]:
# merged_data = pd.concat([same_data, diff_data])
# temp = pd.merge(merged_data, features_data, left_on='img_id_A', right_on='img_id', how='left')
# temp = pd.merge(temp, features_data, left_on='img_id_B', right_on='img_id', how='left', suffixes = ('_a', '_b'))
# temp['f1'] = abs(temp['f1_a']-temp['f1_b'])
# temp['f2'] = abs(temp['f2_a']-temp['f2_b'])
# temp['f3'] = abs(temp['f3_a']-temp['f3_b'])
# temp['f4'] = abs(temp['f4_a']-temp['f4_b'])
# temp['f5'] = abs(temp['f5_a']-temp['f5_b'])
# temp['f6'] = abs(temp['f6_a']-temp['f6_b'])
# temp['f7'] = abs(temp['f7_a']-temp['f7_b'])
# temp['f8'] = abs(temp['f8_a']-temp['f8_b'])
# temp['f9'] = abs(temp['f9_a']-temp['f9_b'])
# temp = temp.drop(columns = ['img_id_a', 'img_id_b', 'Unnamed: 0_a', 'Unnamed: 0_b', ])
# temp = temp.drop(columns = ['f1_a','f2_a','f3_a','f4_a','f5_a','f6_a','f7_a','f8_a','f9_a'])
# temp = temp.drop(columns = ['f1_b','f2_b','f3_b','f4_b','f5_b','f6_b','f7_b','f8_b','f9_b'])
# print(temp.shape, temp.columns.values, temp.loc[1,:])


In [6]:
feature_cat = {'hod':''}

In [101]:
# print(merged_data.iloc[:,2])
temp1 = pd.merge(merged_data, features_data, left_on='img_id_A', right_on='img_id', how='left')
temp2 = pd.merge(merged_data, features_data, left_on='img_id_B', right_on='img_id', how='left')
raw_data = abs(temp1.iloc[:,5:] - temp2.iloc[:,5:])
raw_target = merged_data['target']
print(raw_data[1:5])
print(raw_target.iloc[1:5])
# print(merged_data.iloc[1], feature_diff.iloc[1])
# target = temp1.loc['target']
# print(target)
# merged_data.iloc[3:] = feature_diff
# print(merged_data.iloc[1])
# print(temp1.columns.values)

   f1  f2  f3  f4  f5  f6  f7  f8  f9
1   0   0   1   3   0   0   1   1   0
2   1   0   0   1   0   1   0   1   0
3   1   0   0   3   0   0   0   0   0
4   0   0   0   3   0   0   0   2   0
1    1
2    1
3    1
4    1
Name: target, dtype: int64
